In [253]:
# import the necessary libraries you need for your analysis
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd 
%matplotlib inline

# warnings supression
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [254]:
# read in csv file and display first 5 rows of datset
df = pd.read_excel("data/MuesliProject_raw_data.xlsx")

In [255]:
# read in csv file and display first 5 rows of datset
df_campaign = pd.read_excel("data/MuesliProject_raw_data.xlsx",sheet_name='Campaign Data')
df_process = pd.read_excel("data/MuesliProject_raw_data.xlsx",sheet_name='Order Process Data')
df_intern = pd.read_excel("data/MuesliProject_raw_data.xlsx",sheet_name='InternData Study')

In [256]:
#column cleaning
headers = df.iloc[0]
df.rename(columns=headers, inplace=True)
df.drop(df.index[0], inplace=True)

#drop the new index column
df = df.reset_index(col_level=0)
df.drop('index',axis=1,inplace=True)

In [257]:
#change the column description - underscore instead of space and all str lower
cols = df.columns.tolist()
df.columns = [col.replace(' ','_').lower()for col in cols]

cols = df_campaign.columns.tolist()
df_campaign.columns = [col.replace(' ','_').lower()for col in cols]

cols = df_process.columns.tolist()
df_process.columns = [col.replace(' ','_').lower()for col in cols]

cols = df_intern.columns.tolist()
df_intern.columns = [col.replace(' ','_').lower()for col in cols]


In [258]:
#drop the duplicates from these tables
df_campaign.drop_duplicates(subset='order_id', keep='first' ,inplace=True, ignore_index=False)
df_process.drop_duplicates(subset='order_id', keep='first' ,inplace=True, ignore_index=False)
df_intern.drop_duplicates(subset='order_id', keep='first' ,inplace=True, ignore_index=False)
df.drop_duplicates(subset='order_id', keep='first' ,inplace=True, ignore_index=False)

In [259]:
#merge all tables
df_arrival= df.merge(df_campaign, how='left' , on='order_id').merge(df_process,how='inner', on='order_id').merge(df_intern,how='left', on='order_id')

In [260]:
#only take the column that are needed
df_arrival_final = df_arrival[["order_id", "order_date_x", "ready_to_ship_date", "pickup_date", "on_truck_scan_date", "arrival_scan_date", "ship_mode_y", "ship_mode_x"]]

In [261]:
#the final order process table
df_order_process_final = df_arrival_final[["order_id", "order_date_x", "on_truck_scan_date" ,"ship_mode_y", "ship_mode_x"]]
df_order_process_final.head()

,order_id,order_date_x,on_truck_scan_date,ship_mode_y,ship_mode_x
0,CA-2019-121755,2019-01-16 00:00:00,2019-01-23,Standard Processing,Second Class
1,CA-2019-118255,2019-03-11 00:00:00,2019-03-13,Express,First Class
2,CA-2019-169194,2019-06-20 00:00:00,2019-06-26,Standard Processing,Standard Class
3,CA-2019-111682,2019-06-17 00:00:00,2019-06-19,Express,First Class
4,CA-2020-155558,2020-10-26 00:00:00,2020-11-04,Standard Processing,Standard Class


In [262]:
#the final campaign table
df_campaign_final = df_order_process_final.merge(df_campaign, how='inner' , on='order_id')
df_campaign_final.head()

,order_id,order_date_x,on_truck_scan_date,ship_mode_y,ship_mode_x,arrival_scan_date,customer_name
0,CA-2019-145625,2019-09-11 00:00:00,2019-09-18,Standard Processing,Standard Class,2019-09-23,Kelly Collister
1,CA-2019-162138,2019-04-23 00:00:00,2019-05-01,Standard Processing,Standard Class,2019-05-06,Grace Kelly
2,CA-2019-169166,2019-05-09 00:00:00,2019-05-15,Standard Processing,Standard Class,2019-05-20,Sonia Sunley
3,CA-2020-131954,2020-01-21 00:00:00,2020-01-29,Standard Processing,Standard Class,2020-02-03,Darrin Sayre
4,CA-2019-102981,2019-09-06 00:00:00,2019-09-11,Standard Processing,Second Class,2019-09-16,Mary O'Rourke


In [263]:
#the final intern table
df_intern_final = df_order_process_final.merge(df_intern, how='inner' , on='order_id')
df_intern_final.head() 

,order_id,order_date_x,on_truck_scan_date,ship_mode_y,ship_mode_x,ready_to_ship_date,pickup_date
0,CA-2020-119004,2020-11-23 00:00:00,2020-12-02,Standard Processing,Standard Class,2020-11-30,2020-12-02
1,CA-2019-145625,2019-09-11 00:00:00,2019-09-18,Standard Processing,Standard Class,2019-09-17,2019-09-18
2,CA-2020-135307,2020-11-26 00:00:00,2020-11-27,Express,First Class,2020-11-27,2020-11-27
3,CA-2019-102981,2019-09-06 00:00:00,2019-09-11,Standard Processing,Second Class,2019-09-09,2019-09-11
4,US-2020-155425,2020-11-10 00:00:00,2020-11-12,Express,First Class,2020-11-11,2020-11-12


In [264]:
#to cut out the weekend and only have weekdays
df_intern_final["work_days_to_process"]=np.busday_count(df_intern_final ['order_date_x'].values.astype('datetime64[D]'), df_intern_final ['ready_to_ship_date'].values.astype('datetime64[D]'), weekmask='Mon Tue Wed Thu Fri')

In [265]:
#to cut out the weekend and only have weekdays
df_intern_final["work_days_to_truck"]=np.busday_count(df_intern_final ['order_date_x'].values.astype('datetime64[D]'), df_intern_final ['on_truck_scan_date'].values.astype('datetime64[D]'), weekmask='Mon Tue Wed Thu Fri')

In [266]:
df_intern_final['waiting_for_truck'] = df_intern_final['work_days_to_truck'] - df_intern_final['work_days_to_process']

In [278]:
df_intern_final = df_intern_final[["order_id","order_date_x","on_truck_scan_date","ship_mode_y","ship_mode_x","ready_to_ship_date","pickup_date","work_days_to_process","waiting_for_truck","work_days_to_truck"]]

In [268]:
#to cut out the weekend and only have weekdays
df_campaign_final["total_delivery_days"]=np.busday_count(df_campaign_final ['order_date_x'].values.astype('datetime64[D]'), df_campaign_final ['arrival_scan_date'].values.astype('datetime64[D]'), weekmask='Mon Tue Wed Thu Fri')

In [ ]:
df_campaign_final['total_delivery_days_w_weekend'] = (df_campaign_final['arrival_scan_date'] - df_campaign_final['order_date_x']).dt.days 

In [270]:
df_campaign_final["week_day_arrival"] =df_campaign_final["arrival_scan_date"].dt.day_name()

In [271]:
df_campaign_final['order_date_x'] = pd.to_datetime(df_campaign_final['order_date_x'])
df_campaign_final["week_day_arrival"] =df_campaign_final["arrival_scan_date"].dt.day_name()
df_campaign_final["week_day_order"] =df_campaign_final["order_date_x"].dt.day_name()


In [280]:
df_campaign_final['how_many_weekends'] = ((df_campaign_final['total_delivery_days_w_weekend'] - df_campaign_final['total_delivery_days'])/2)
df_campaign_final['how_many_weekends'] = df_campaign_final['how_many_weekends'].astype('int64')

In [282]:
df_campaign_final.head(3)

,order_id,order_date_x,on_truck_scan_date,ship_mode_y,ship_mode_x,arrival_scan_date,customer_name,total_delivery_days,total_delivery_days_w_weekend,week_day_arrival,week_day_order,how_many_weekends
0,CA-2019-145625,2019-09-11,2019-09-18,Standard Processing,Standard Class,2019-09-23,Kelly Collister,8,12,Monday,Wednesday,2
1,CA-2019-162138,2019-04-23,2019-05-01,Standard Processing,Standard Class,2019-05-06,Grace Kelly,9,13,Monday,Tuesday,2
2,CA-2019-169166,2019-05-09,2019-05-15,Standard Processing,Standard Class,2019-05-20,Sonia Sunley,7,11,Monday,Thursday,2


In [292]:
pd.crosstab(index=df_intern_final["ship_mode_y"],columns=df_intern_final["work_days_to_process"], normalize="index")\
    .round(4)*100

work_days_to_process,0,1,2,3,4,5
ship_mode_y,,,,,,
Express,33.96,15.09,28.30,22.64,0.00,0.00
Standard Processing,2.65,2.65,19.87,29.80,31.79,13.25
